根据现有模型，遍历整个图，输出某些信息

In [1]:
import os
import torch as t
import tqdm

from data_preprocess_cv import get_data, CancerDataset
import config_load
from main import *
from utils import get_node_name

configs = config_load.get()
gpu = f"cuda:0"
configs["device"] = gpu
configs['load_data'] = True
configs["log_name"] = f"test"
configs["logfile"] = os.path.join(configs["log_dir"], configs["log_name"] + ".txt")
configs["batch_size"] = 2000
# configs["hic"] = False
# configs['data_dir'] = "data/Leukemia_Matrix/"

dataset = get_data(configs=configs, stable=True)
ckpt_path = "./predict/models/MCF7_Hi-C/" if configs['hic'] else './predict/models/MCF7_PPI/'
checkpoints = os.listdir(ckpt_path)
train_result = []
for i in range(10):
    configs["fold"] = i
    modules = get_training_modules(configs, dataset, pred=True)
    ckpt = ckpt_path + checkpoints[i]
    # ckpt = t.load(ckpt)['state_dict']
    y_score, y_pred, y_true, y_index, edge_weight_1, edge_weight_2, edge_index_1, edge_index_2 = predict(modules['model'], modules['train_loader_list'], configs, ckpt)
    edge_attention = pd.DataFrame(columns=['Gene 1', 'Label 1', 'Gene 2', 'Label 2', 'Attention 1-1', 'Attention 1-2', 'Attention 1-3', 'Attention 2-1', 'Attention 2-2', 'Attention 2-3'])
    # print(edge_weight_1)
    for k in range(len(edge_weight_1)):
        for j in tqdm(range(len(edge_weight_1[k]))):
            new_row = pd.DataFrame({
                'Gene 1' : [y_index[edge_index_1[k][1][j].item()]],
                'Gene 2' : [y_index[edge_index_1[k][0][j].item()]],
                'Attention 1-1' : [edge_weight_1[k][j][0].item()],
                'Attention 1-2' : [edge_weight_1[k][j][1].item()],
                'Attention 1-3' : [edge_weight_1[k][j][2].item()],
                'Attention 2-1' : [edge_weight_2[k][j][0].item()],
                'Attention 2-2' : [edge_weight_2[k][j][1].item()],
                'Attention 2-3' : [edge_weight_2[k][j][2].item()],
                })
            edge_attention = pd.concat([edge_attention, new_row], ignore_index=True)
            # if j == 100: break
        # break
    edge_attention['Gene 1'] = get_node_name(edge_attention['Gene 1'].tolist())
    edge_attention['Gene 2'] = get_node_name(edge_attention['Gene 2'].tolist())
    edge_attention.to_excel(f"data/Results/Edge_Attention_{i}.xlsx", index=False)
    # print(edge_attention)

    # acc, cf_matrix, auprc, f1, auc = calculate_metrics(y_true, y_pred, y_score)
    # tp = cf_matrix[1, 1]
    # with open(configs['logfile'], 'a') as f:
    #     print("Test AUPRC:{:.4f}, AUROC:{:.4f}, ACC:{:.4f}, F1:{:.4f}, TP:{:.1f}"
    #         .format(auprc, auc, acc, f1, tp), file=f, flush=True)
    # train_result =  pred_to_df(i, train_result, y_index, y_true, y_score)
# score_col = [f"score_{i}" for i in range(10)]
# train_result['avg_score'] = train_result[score_col].mean(axis=1)
# train_result['pred_label'] = train_result.apply(
#     lambda x: 1 if x['avg_score'] > 0.5 else 0, axis=1)
# score_avg_perfomance(train_result, score_col, configs['logfile'])

Loading dataset from: data/Breast_Cancer_Matrix/MCF7_CPDB_dataset_final.pkl ......
Doing below distrubs: 
 {'add': [], 'remove': [], 'random': []}
Drop  0.0 of negative train samples and  0.0 of positive train samples
Loading model from ./predict/models/MCF7_Hi-C/325_0.9264_24.pkl ......


 27%|██▋       | 145653/547114 [34:37<1:35:25, 70.12it/s]


KeyboardInterrupt: 